In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [ ]:
# from utils.df_handle import *
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")

name='IV'
prefix='SYNC_'
csv_path = '/usr/local/airflow/plugins'+'/'

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 5, 10, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '*/30 8-17,23-23 * * *',
          tags=[prefix+name, 'Sync', '30mins']
)

In [2]:
start_date = '2022-01-01'
datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y%m%d")
datenow_mns45 = (datetime.now() - timedelta(days=45)).strftime("%Y-%m-%d")

In [3]:
from_tb = "OM_Invoice"

In [15]:
table_name = "sync_dms_iv"

In [21]:
usql = \
f"""
DECLARE @from DATE = '{datenow}'
DECLARE @to DATE = '2022-01-31'
SELECT
CONCAT(BranchID,RefNbr) as pk,
BranchID,
RefNbr,
InvcNbr,
InvcNote,
Crtd_DateTime,
LUpd_User,
LUpd_DateTime
from {from_tb}
where LUpd_DateTime >= @from
"""

In [24]:
def update():
    try:
        #UPDATE
        df_update = get_ms_df(usql)
        assert df_update.shape[0] >0,"NO DATA TO INPUT"
        df_update.columns = lower_col(df_update)
        # df_update.columns
        df_update['crtd_datetime'] = df_update.crtd_datetime.dt.normalize()
        df_update1 = df_update['crtd_datetime']
        df_update1.drop_duplicates(inplace=True)
        tpl_dt = tuple(df_update1.dt.strftime('%Y-%m-%d').to_list()) + ('1900-01-01',)
        # tpl_dt
        df_update1 = df_update['pk']
        df_update1.drop_duplicates(inplace=True)
        tpl_pk = tuple(df_update1.to_list()) + ('',)
        del_sql = \
        f"""
        DELETE FROM biteam.{table_name}
        WHERE
        DATE(crtd_datetime) in {tpl_dt}
        AND pk in {tpl_pk}
        """
        print("del_sql ",del_sql)
        execute_bq_query(del_sql)
        df_update['inserted_at'] = datetime.now()
        bq_values_insert(df_update, f"{table_name}", 2)
    except AssertionError:
        print("NO DATA TO INPUT")

del_sql  
DELETE FROM biteam.sync_dms_iv
WHERE
DATE(crtd_datetime) in ('2022-05-15', '1900-01-01')
AND pk in ('MR0013EP0000701', 'MR0013EP0000702', 'MR0013EP0000703', 'MR0012IN0016218', 'MR0012IN0016219', 'MR0012IN0016220', 'MR0012IN0016221', 'MR0001IN0098508', 'MR0001IN0098509', 'MR0001IN0098510', 'MR0001IN0098523', 'MR0001IN0098544', 'MR0001IN0098545', 'MR0001IN0098548', 'MR0001IN0098551', 'MR0014IN0021451', 'MR0014IN0021452', 'MR0014IN0021453', 'MR0014IN0021454', 'MR0014IN0021455', 'MR0014IN0021456', 'MR0014IN0021457', 'MR0014IN0021458', 'MR0014IN0021459', 'MR0014IN0021460', 'MR0014IN0021461', 'MR0010IN0084081', 'MR0010IN0084082', 'MR0010IN0084083', 'MR0010IN0084084', 'MR0010IN0084085', 'MR0010IN0084086', 'MR0010IN0084087', 'MR0010IN0084088', 'MR0010IN0084089', 'MR0010IN0084090', 'MR0016IN0041395', 'MR0015IN0043545', 'MR0015IN0043546', 'MR0015IN0043547', 'MR0015IN0043548', 'MR0015IN0043549', 'MR0015IN0043550', 'MR0015IN0043551', 'MR0015IN0043552', '')



1it [00:02,  2.55s/it]


In [ ]:
# dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

# update = PythonOperator(task_id="update", python_callable=update, dag=dag)

# dummy_start >> update